# Stock API
Using [Yahoo Finance API](https://python-yahoofinance.readthedocs.io/en/latest/api.html) to get stock information.

# Import Libraries

In [ ]:
!pip install fastai

     |████████████████████████████████| 186 kB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 127.9 MB 82.9 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 12.6 MB/s eta 0:00:01
     |████████████████████████████████| 13.9 MB 16.9 MB/s eta 0:00:01
     |████████████████████████████████| 56 kB 13.4 MB/s eta 0:00:01
     |████████████████████████████████| 450 kB 74.7 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 19.0 MB/s eta 0:00:01


In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import math
from os import walk

import yfinance as yf
from ta import add_all_ta_features
import fastai.tabular

ModuleNotFoundError: No module named 'fastai'

# Data

In [54]:
path = "Sample Data/"
filenames = next(walk(path), (None, None, []))[2]
print("Number of Stocks:", len(filenames))

Number of Stocks: 43


In [55]:
tickerSymbols = [file.split("_", 1)[0] for file in filenames]
tickerSymbols[0:5]

['TSLA', 'AAPL', 'JNJ', 'JPM', 'AMZN']

In [56]:
index = 0
tickerSymbol = tickerSymbols[index]
tickerSymbol

'TSLA'

In [57]:
stock = pd.read_csv(path + filenames[index])
stock

,date,symbol,open,high,low,close,volume,turnover,preclose,risefall,risefallratio
0,2021-01-29 00:00:00,TSLA,830.00,842.41,780.10,793.53,2.817083e+10,0,835.43,-41.90,-5.02
1,2021-01-28 00:00:00,TSLA,820.00,848.00,801.00,835.43,2.193008e+10,0,864.16,-28.73,-3.32
2,2021-01-27 00:00:00,TSLA,870.35,891.50,858.66,864.16,2.375123e+10,0,883.09,-18.93,-2.14
3,2021-01-26 00:00:00,TSLA,891.38,895.90,871.60,883.09,2.052623e+10,0,880.80,2.29,0.26
4,2021-01-25 00:00:00,TSLA,855.00,900.40,838.82,880.80,3.611176e+10,0,846.64,34.16,4.03
...,...,...,...,...,...,...,...,...,...,...,...
95,2020-09-14 00:00:00,TSLA,380.95,420.00,373.30,419.62,3.330614e+10,0,372.72,46.90,12.58
96,2020-09-11 00:00:00,TSLA,381.94,382.50,360.50,372.72,2.248211e+10,0,371.34,1.38,0.37
97,2020-09-10 00:00:00,TSLA,386.21,398.99,360.56,371.34,3.244167e+10,0,366.28,5.06,1.38
98,2020-09-09 00:00:00,TSLA,356.60,369.00,341.51,366.28,2.825391e+10,0,330.21,36.07,10.92


In [58]:
stock['date']  = pd.to_datetime(stock['date']).astype(str)
stock['date']
endDate = max(stock["date"])
startDate = min(stock["date"])
print("Date range:", startDate, "to", endDate)

Date range: 2020-09-08 to 2021-01-29


In [59]:
# Create ticker yfinance object
tickerData = yf.Ticker(tickerSymbol)

# Create historic data dataframe and fetch the data for the dates given. 
df = tickerData.history(start = startDate, end = endDate)

# The dataframe has:
#   - Date (YYY-MM-DD) as an index
#   - Open (price the stock started as)
#   - High (highest price stock reached that day)
#   - Low (lowest price stock reached that day)
#   - Close (price the stock ended the day as)
#   - Volume (how many shares were traded that day)
#   - Dividends (any earnings shared to shareholders)
#   - Stock Splits (any stock price changes)

print(df.head())

                  Open       High         Low       Close     Volume  \
Date                                                                   
2020-09-08  356.000000  368.73999  329.880005  330.209991  115465700   
2020-09-09  356.600006  369.00000  341.510010  366.279999   79465800   
2020-09-10  386.209991  398.98999  360.559998  371.339996   84930600   
2020-09-11  381.940002  382.50000  360.500000  372.720001   60717500   
2020-09-14  380.950012  420.00000  373.299988  419.619995   83020600   

            Dividends  Stock Splits  
Date                                 
2020-09-08          0             0  
2020-09-09          0             0  
2020-09-10          0             0  
2020-09-11          0             0  
2020-09-14          0             0  


In [60]:
# Change the date column to a pandas date time column 

# Define string format
date_change = '%Y-%m-%d'

# Create a new date column from the index
df['Date'] = df.index

# Perform the date type change
df['Date'] = pd.to_datetime(df['Date'], format = date_change)

# Create a variable that is the date column
Dates = df['Date']

In [61]:
# Add all technical analysis to the dataframe we've already loaded
df = add_all_ta_features(df, "Open", "High", "Low", "Close", "Volume", fillna=True) 

print(df.columns)

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'Date', 'volume_adi', 'volume_obv', 'volume_cmf', 'volume_fi',
       'volume_mfi', 'volume_em', 'volume_sma_em', 'volume_vpt', 'volume_nvi',
       'volume_vwap', 'volatility_atr', 'volatility_bbm', 'volatility_bbh',
       'volatility_bbl', 'volatility_bbw', 'volatility_bbp', 'volatility_bbhi',
       'volatility_bbli', 'volatility_kcc', 'volatility_kch', 'volatility_kcl',
       'volatility_kcw', 'volatility_kcp', 'volatility_kchi',
       'volatility_kcli', 'volatility_dcl', 'volatility_dch', 'volatility_dcm',
       'volatility_dcw', 'volatility_dcp', 'volatility_ui', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow', 'trend_adx',
       'trend_adx_pos', 'trend_adx_neg', 'trend_vortex_ind_pos',
       'trend_vortex_ind_neg', 'trend_vortex_ind_diff', 'trend_trix',
       'trend_mass_index', 'trend_cci', 'trend_

/Users/simplypatrickg/anaconda3/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/Users/simplypatrickg/anaconda3/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
